# Dictionary Operations with FolderDB

This notebook demonstrates how to work with nested dictionaries using the FolderDB class. We'll show:
- Creating and storing nested dictionaries
- Updating specific values in nested structures
- Querying dictionary data
- Deleting dictionary entries

## Setup and Imports

First, let's import the required libraries and set up our environment.

In [2]:
import os
import json
from jsonldb.folderdb import FolderDB

## Initialize Database

Let's create a folder for our database and initialize the FolderDB instance.

In [3]:
# Create a folder for our database
db_folder = "dict_db"
os.makedirs(db_folder, exist_ok=True)

# Initialize the database
db = FolderDB(db_folder)

## Create Sample Data

Let's create some sample nested dictionaries for products and inventory.

In [4]:
# Create products dictionary
products = {
    "prod1": {
        "name": "Laptop",
        "price": 1000,
        "specs": {
            "cpu": "Intel i7",
            "ram": "16GB",
            "storage": "512GB SSD"
        }
    },
    "prod2": {
        "name": "Smartphone",
        "price": 500,
        "specs": {
            "cpu": "Snapdragon 8",
            "ram": "8GB",
            "storage": "256GB"
        }
    }
}

# Create inventory dictionary
inventory = {
    "loc1": {
        "name": "Warehouse A",
        "address": "123 Main St",
        "stock": {
            "prod1": 10,
            "prod2": 20
        }
    },
    "loc2": {
        "name": "Warehouse B",
        "address": "456 Oak Ave",
        "stock": {
            "prod1": 5,
            "prod2": 15
        }
    }
}

print("Products Dictionary:")
print(json.dumps(products, indent=2))
print("\nInventory Dictionary:")
print(json.dumps(inventory, indent=2))

Products Dictionary:
{
  "prod1": {
    "name": "Laptop",
    "price": 1000,
    "specs": {
      "cpu": "Intel i7",
      "ram": "16GB",
      "storage": "512GB SSD"
    }
  },
  "prod2": {
    "name": "Smartphone",
    "price": 500,
    "specs": {
      "cpu": "Snapdragon 8",
      "ram": "8GB",
      "storage": "256GB"
    }
  }
}

Inventory Dictionary:
{
  "loc1": {
    "name": "Warehouse A",
    "address": "123 Main St",
    "stock": {
      "prod1": 10,
      "prod2": 20
    }
  },
  "loc2": {
    "name": "Warehouse B",
    "address": "456 Oak Ave",
    "stock": {
      "prod1": 5,
      "prod2": 15
    }
  }
}


## Save Data to Database

Now let's save our dictionaries to the database using the `upsert_dicts` method.

In [5]:
# Save dictionaries to database
db.upsert_dicts({
    "products": products,
    "inventory": inventory
})

print("Database state after saving:")
print(str(db))

Database state after saving:
FolderDB at dict_db
--------------------------------------------------
Found 2 JSONL files

products.jsonl:
  Size: 208 bytes
  Count: 2
  Key range: prod1 to prod2
  Linted: False

inventory.jsonl:
  Size: 175 bytes
  Count: 2
  Key range: loc1 to loc2
  Linted: False




## Update Records

Let's update the price of a product using the `upsert_dict` method.

In [6]:
# Update product price
updated_product = {
    "prod1": {
        "name": "Laptop",
        "price": 1200,  # Updated price
        "specs": {
            "cpu": "Intel i7",
            "ram": "16GB",
            "storage": "512GB SSD"
        }
    }
}

db.upsert_dict("products", updated_product)

print("Updated product information:")
print(json.dumps(db.get_dict(["products"])["products"], indent=2))

Updated product information:
{
  "prod1": {
    "name": "Laptop",
    "price": 1200,
    "specs": {
      "cpu": "Intel i7",
      "ram": "16GB",
      "storage": "512GB SSD"
    }
  },
  "prod2": {
    "name": "Smartphone",
    "price": 500,
    "specs": {
      "cpu": "Snapdragon 8",
      "ram": "8GB",
      "storage": "256GB"
    }
  }
}


## Query Records

Let's demonstrate different ways to query dictionary data from the database.

In [7]:
# Query all products
print("All products:")
print(json.dumps(db.get_dict(["products"])["products"], indent=2))

# Query specific warehouse inventory
print("\nWarehouse A inventory:")
print(json.dumps(db.get_dict(["inventory"], lower_key="loc1", upper_key="loc1")["inventory"], indent=2))

All products:
{
  "prod1": {
    "name": "Laptop",
    "price": 1200,
    "specs": {
      "cpu": "Intel i7",
      "ram": "16GB",
      "storage": "512GB SSD"
    }
  },
  "prod2": {
    "name": "Smartphone",
    "price": 500,
    "specs": {
      "cpu": "Snapdragon 8",
      "ram": "8GB",
      "storage": "256GB"
    }
  }
}

Warehouse A inventory:
{
  "loc1": {
    "name": "Warehouse A",
    "address": "123 Main St",
    "stock": {
      "prod1": 10,
      "prod2": 20
    }
  }
}


## Update Nested Data

Let's update the inventory for a specific location.

In [8]:
# Update warehouse inventory
updated_inventory = {
    "loc1": {
        "name": "Warehouse A",
        "address": "123 Main St",
        "stock": {
            "prod1": 15,  # Updated quantity
            "prod2": 25   # Updated quantity
        }
    }
}

db.upsert_dict("inventory", updated_inventory)

print("Updated inventory:")
print(json.dumps(db.get_dict(["inventory"])["inventory"], indent=2))

Updated inventory:
{
  "loc1": {
    "name": "Warehouse A",
    "address": "123 Main St",
    "stock": {
      "prod1": 15,
      "prod2": 25
    }
  },
  "loc2": {
    "name": "Warehouse B",
    "address": "456 Oak Ave",
    "stock": {
      "prod1": 5,
      "prod2": 15
    }
  }
}


## Delete Records

Let's delete a warehouse location from the inventory.

In [9]:
# Delete warehouse location
db.delete_file_keys("inventory", ["loc2"])

print("Database state after deletion:")
print(str(db))

print("\nRemaining inventory:")
print(json.dumps(db.get_dict(["inventory"])["inventory"], indent=2))

Database state after deletion:
FolderDB at dict_db
--------------------------------------------------
Found 2 JSONL files

products.jsonl:
  Size: 208 bytes
  Count: 2
  Key range: prod1 to prod2
  Linted: False

inventory.jsonl:
  Size: 175 bytes
  Count: 1
  Key range: loc1 to loc1
  Linted: False



Remaining inventory:
{
  "loc1": {
    "name": "Warehouse A",
    "address": "123 Main St",
    "stock": {
      "prod1": 15,
      "prod2": 25
    }
  }
}


## Lint DB

In [10]:
db.lint_db()

Found 2 JSONL files to lint.
Linting file: products.jsonl
Successfully linted and updated metadata for products.jsonl.
Linting file: inventory.jsonl
Successfully linted and updated metadata for inventory.jsonl.


In [11]:
print(db)

FolderDB at dict_db
--------------------------------------------------
Found 2 JSONL files

products.jsonl:
  Size: 208 bytes
  Count: 2
  Key range: prod1 to prod2
  Linted: True

inventory.jsonl:
  Size: 88 bytes
  Count: 1
  Key range: loc1 to loc1
  Linted: True




## Cleanup

Finally, let's clean up by removing the database folder and its contents.

In [12]:
# Cleanup
for file in os.listdir(db_folder):
    os.remove(os.path.join(db_folder, file))
os.rmdir(db_folder)

print("Database folder has been cleaned up.")

Database folder has been cleaned up.
